<a href="https://colab.research.google.com/github/v-y-l/Machine-Learning-Notebooks/blob/main/Victor's_unbiased_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unbiased estimation: Training phi and psi as two neural networks
## Section author: Victor Lin (vl2580)

## Theory

This approach improves on the previous **biased** (Taylor-based) method, whose accuracy is limited by the polynomial degree, no matter how much data we use.

In contrast, this method is **unbiased**, since more data improves results.

### Structure
$$
\mathbf{y} = \text{GELU}(\mathbf{W} \mathbf{x}) \quad \text{with} \quad \mathbf{y}' = \Phi(\mathbf{W}) \Psi(\mathbf{x})
$$

- ψ: MLP mapping $\mathbf{x} \in \mathbb{R}^d \to \mathbb{R}^m$
- φ: row-wise MLP mapping $\mathbf{W} \in \mathbb{R}^{d \times d} \to \mathbb{R}^{d \times m}$
- Final prediction: $\mathbf{y}' = \Phi(\mathbf{W}) \cdot \Psi(\mathbf{x})$ (row-wise dot)

### Training

Train φ and ψ to minimize MSE:

$$
\textbf{L} = \frac{1}{N} \sum_{i=1}^N \left\| \Phi(\mathbf{W}_i) \Psi(\mathbf{x}_i) - \text{GELU}(\mathbf{W}_i \mathbf{x}_i) \right\|^2
$$

### Why it works

This method builds a computation graph across both φ and ψ. It scales with data and model size. After training, we can discard the GELU node and use:

$$
\mathbf{y}' = \Phi(\mathbf{W}) \Psi(\mathbf{x})
$$

as a learned approximation.

## Implementation

In [1]:
# Train φ(W) and ψ(x) as neural networks for an unbiased linearization of GELU(Wx).
# PyTorch builds a computation graph across both nets, joined at the final dot product.
# Mini-batch optimization ensures efficient gradient updates across φ and ψ.

import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader

class GeluDataset(Dataset):
    def __init__(self, N, d):
        self.W = torch.randn(N, d, d)
        self.x = torch.randn(N, d)
        Z = torch.einsum('bij,bj->bi', self.W, self.x)
        self.y = 0.5 * Z * (1 + torch.erf(Z / math.sqrt(2)))

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.W[idx], self.x[idx], self.y[idx]

class PsiNet(nn.Module):
    def __init__(self, d, m, hidden=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, hidden),
            nn.ReLU(),
            nn.Linear(hidden, m)
        )

    def forward(self, x):
        return self.net(x)

class PhiNet(nn.Module):
    def __init__(self, d, m, hidden=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, hidden),
            nn.ReLU(),
            nn.Linear(hidden, m)
        )

    def forward(self, W):
        B, D, _ = W.shape
        W_flat = W.view(B * D, D)
        out_flat = self.net(W_flat)
        return out_flat.view(B, D, -1)

# --- Hyperparameters ---
# d: input/output dimension; m: hidden feature dimension
# N: dataset size; batch_size: training mini-batch size
# epochs: number of full passes over the data
d, m, N, batch_size, epochs = 2, 16, 1024, 64, 5

# --- Setup data + models ---
dataset = GeluDataset(N, d)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
psi = PsiNet(d, m)
phi = PhiNet(d, m)

# --- Optimizer ---
# Adam: fast convergence with minimal tuning, good for deep MLPs
optimizer = torch.optim.Adam(list(psi.parameters()) + list(phi.parameters()), lr=1e-3)
criterion = nn.MSELoss()

# --- Training loop ---
for epoch in range(1, epochs + 1):
    total_loss = 0.0
    for Wb, xb, yb in loader:
        optimizer.zero_grad()
        ψ = psi(xb)                      # (B, m)
        Φ = phi(Wb)                      # (B, d, m)
        y_pred = torch.bmm(Φ, ψ.unsqueeze(-1)).squeeze(-1)  # (B, d)
        loss = criterion(y_pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * Wb.size(0)

    print(f"Epoch {epoch}: MSE = {total_loss / N:.4f}")

Epoch 1: MSE = 0.8032
Epoch 2: MSE = 0.2772
Epoch 3: MSE = 0.1711
Epoch 4: MSE = 0.1388
Epoch 5: MSE = 0.1190


In [4]:
import numpy as np
import torch

class NeuralGELUComparator:
    def __init__(self, phi_model, psi_model):
        self.phi = phi_model
        self.psi = psi_model

    def gelu_tanh(self, x):
        return 0.5 * x * (1 + np.tanh(np.sqrt(2/np.pi) * (x + 0.044715 * x**3)))

    def compare(self, W_test, x_test):
        with torch.no_grad():
            ψ = self.psi(x_test)                         # (1, m)
            Φ = self.phi(W_test)                         # (1, d, m)
            y_approx = torch.bmm(Φ, ψ.unsqueeze(-1)).squeeze().numpy()

            x_proj = torch.bmm(W_test, x_test.unsqueeze(-1)).squeeze().numpy()
            y_true = self.gelu_tanh(x_proj)

        print("==== Neural φ/ψ Linearization of GELU ====\n")
        print(f"x' = Wx = {x_proj}")
        print(f"Learned φ(W) · ψ(x) = {y_approx}")
        print(f"GELU_tanh(x')      = {y_true}")

        rmse = np.sqrt(np.mean((y_approx - y_true) ** 2))
        print(f"\nRMSE: Linearized φ · ψ vs GELU_tanh = {rmse:.5f}")

class NeuralGELUDemo:
    def __init__(self, phi_model, psi_model, d):
        self.phi = phi_model
        self.psi = psi_model
        self.d = d
        self.comparator = NeuralGELUComparator(phi_model, psi_model)

    def run(self):
        W_test = torch.randn(1, self.d, self.d)
        x_test = torch.randn(1, self.d)
        self.comparator.compare(W_test, x_test)

NeuralGELUDemo(phi, psi, d=2).run()

==== Neural φ/ψ Linearization of GELU ====

x' = Wx = [-0.7917412   0.51313776]
Learned φ(W) · ψ(x) = [-0.14708376  0.3033128 ]
GELU_tanh(x')      = [-0.16971655  0.35716217]

RMSE: Linearized φ · ψ vs GELU_tanh = 0.04130
